In [35]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import numpy as np
import matplotlib.pyplot as plt
import sympy as sym
import pandas as pd
import openpyxl

from scipy.stats import norm
from scipy.stats import t
from scipy.stats import chi2

# Resonancia eléctrica

In [36]:
def Minimos_cuadrados(x,y,n=1):
    l = x.shape[0]
    b = y
    
    A = np.ones((l,n+1))
    
    for i in range(1,n+1):
        A[:,i] = x**i
        
    AT = np.dot(A.T,A)
    bT = np.dot(A.T,b)

    xsol = np.linalg.solve(AT,bT)
    
    return xsol

def GetModel(x,p):
    
    y = 0.
    for i in range(len(p)):
        y += p[i]*x**i
        
    return y

def GetError(x,y,p,n=1):
    
    l = x.shape[0]
    
    A = np.ones((l,n+1))
    
    for i in range(1,n+1):
        A[:,i] = x**i
        
    # Residuos
    R = y - np.dot(A,p)
    
    sigma2 = np.dot(R.T,R)/(len(y)-len(p))
    
    Cov = sigma2*np.linalg.inv(np.dot(A.T,A))
    
    R2 = None
    if np.shape(Cov) == (2,2):
        R2 = (Cov[0][1]**2)/(Cov[0][0]*Cov[1][1])

    return (Cov,R2,R)

def Get_sigma(Cov):
    sigmaai = np.sqrt(np.diagonal(Cov))
    return sigmaai[::-1]

def Error_relativo(X_real,X_exp):
    return (np.abs(X_exp - X_real)/X_real)

def presicion(d,v):
    p = (d/v)
    if p <= 0.1:
        return p,True
    else:
        return p,False
    
def exactitud(xr,xm,d):
    e = (np.abs(xr-xm))
    if e <= (2.5*d):
        return e/d,True
    else:
        return e/d,False
    
X = sym.Symbol('x',real=True)
Y = sym.Symbol('y',real=True)

# Datos

In [37]:
# El osciloscopio presentó error a partir de 9kHz
wb = openpyxl.load_workbook(r"C:\Users\ASUS\OneDrive\Uniandes\Física\Ondas y Fluidos\Lab_OyF\Resonancia RLC/resonancia.xlsx")
datos = pd.read_excel(io = r"C:\Users\ASUS\OneDrive\Uniandes\Física\Ondas y Fluidos\Lab_OyF\Resonancia RLC/resonancia.xlsx", header=0,sheet_name='Hoja1',names=None,index_col=None,usecols='A:D',engine='openpyxl')
print(datos.head())

   Frecuencia (kHz)  fase  Vpp1  Vpp2
0               5.5 -40.0  4.72  3.60
1               6.0 -30.0  4.64  3.84
2               6.5 -25.0  4.56  4.08
3               7.0 -15.0  4.56  4.24
4               7.5   0.0  4.48  4.32


In [38]:
datos_ = datos.values
datos_np = np.float64(np.transpose(datos_))
frec = datos_np[0]
fase = datos_np[1]
Vpp1 = datos_np[2]
Vpp2 = datos_np[3]